# 4. Trabajando con Datos Númericos

## Librerias

In [1]:
import pandas as pd
import numpy as np

In [34]:
from sklearn import preprocessing

## 4.1 Reescalar features

In [3]:
feature = np.array([
    [-500.5],
    [-100.1],
    [0],
    [100.1],
    [900.9]
])

In [4]:
feature

array([[-500.5],
       [-100.1],
       [   0. ],
       [ 100.1],
       [ 900.9]])

In [5]:
# Creamos el escalador

minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))

In [6]:
# Escalamos nuestros datos

scaled_feature = minmax_scale.fit_transform(feature)
scaled_feature

array([[0.        ],
       [0.28571429],
       [0.35714286],
       [0.42857143],
       [1.        ]])

## 4.2 Estandarizar features

In [17]:
feature = np.array([
    [-1000.1],
    [-200.2],
    [500.5],
    [600.6],
    [9000.9]
])

In [19]:
# Creamos el escalador

scaler = preprocessing.StandardScaler()

In [20]:
standarized = scaler.fit_transform(feature)
standarized

array([[-0.76058269],
       [-0.54177196],
       [-0.35009716],
       [-0.32271504],
       [ 1.97516685]])

In [24]:
print(f'Mean: {round(standarized.mean())}')
print(f'D. Est: {round(standarized.std())}')

Mean: 0
D. Est: 1


Si nuestros datos tienen outliers significativos podemos usar un escalador robusto

In [25]:
robust_scaler = preprocessing.RobustScaler()

In [26]:
robust_scaler.fit_transform(feature)

array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

## 4.3 Normalizar observaciones

**Normalizar:**  Consiste en ajustar los valores medidos en diferentes escalas respecto a una escala común.

In [27]:
feature = np.array([
    [0.5, 0.5],
    [1.1, 3.4],
    [1.5, 20.2],
    [1.63, 34.4],
    [10.9, 3.3]
])

In [36]:
feature

array([[ 0.5 ,  0.5 ],
       [ 1.1 ,  3.4 ],
       [ 1.5 , 20.2 ],
       [ 1.63, 34.4 ],
       [10.9 ,  3.3 ]])

In [35]:
# Creamos el normalizador 

normalizer = preprocessing.Normalizer(norm='l2')

In [37]:
normalizer.transform(feature)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])